# Report Creator

modifies original data in form of .csv file exported from the system to an excel format with required column naming which is used for weekly reporting purposes

In [16]:
import pandas as pd
import numpy as np
from tkinter import Tk 
from tkinter.filedialog import askopenfilename, askdirectory
import tkinter.messagebox
import tkinter, os
import re
import datetime



root=tkinter.Tk()
root.withdraw()
root.update()
sirus_csv = askopenfilename(title='SELECT CSV FILE FROM SIRUS')# show an "Open" dialog box and return the path to the selected file
#sirus_data = pd.read_csv(sirus_csv,encoding="ISO-8859-1")
now = datetime.datetime.now()
current_time = now.strftime("_%H_%M_%d-%m-%Y")
input_directory=os.path.dirname(sirus_csv)
input_file_name=os.path.splitext(os.path.basename(sirus_csv))[0]
sirus_data=pd.read_csv(os.path.abspath(sirus_csv))

p_date=[]
NAM_date=[]


sirus_data=sirus_data[(sirus_data['Created by']== 'Aleksandra.Czekaj') | (sirus_data['Created by']== 'Monish.Iyappan')]
sirus_data=sirus_data[['Title','Published date','Author','Well','Field','Country','Identifiers','Product type','URL']]
sirus_data.rename(columns={"Well": "Asset", "Identifiers": "Company tag"},inplace=True)


well_is_null=sirus_data['Asset'].isnull()
sirus_data.loc[well_is_null,'Asset']=sirus_data['Field']

well_is_null=sirus_data['Asset'].isnull()
sirus_data.loc[well_is_null,'Asset']=sirus_data['Country']


for date in sirus_data['Published date']:
    date=datetime.datetime.strptime(date, '%d-%b-%Y').strftime('%Y-%m-%d')
    p_date.append(date)

sirus_data['Published date']=p_date

#EXCEPTIONS
data_SUKEP=sirus_data[(sirus_data['Country']== 'United Kingdom')&~(sirus_data['Field']== 'CLIPPER') 
                   &~(sirus_data['Field']== 'CUTTER') 
                      &~(sirus_data['Field']== 'GALLEON') 
                        &~(sirus_data['Field']== 'BARQUE')]

#data_SUKEP['Title']=data_SUKEP['Title'].astype(str) + " " + data_SUKEP['Published date']

data_NAM=sirus_data[(sirus_data['Country']== 'Netherlands') 
                   | (sirus_data['Field']== 'CLIPPER') 
                   | (sirus_data['Field']== 'CUTTER') 
                      | (sirus_data['Field']== 'GALLEON') 
                        | (sirus_data['Field']== 'BARQUE')]


for date in data_NAM['Published date']:
    date=datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%d-%m-%Y')
    NAM_date.append(date)

data_NAM['Published date']=NAM_date
#data_NAM['Title']=data_NAM['Title'].astype(str) + " " + data_NAM['Published date']

data_SUKEP.drop(columns=['Field','Country'],inplace=True)
data_NAM.drop(columns=['Field','Country'],inplace=True)


#create output file name based on the input file name and time
output_file_name_1=input_file_name+'_NAM_'+current_time+'.xlsx'
output_file_name_2=input_file_name+'_SUKEP_'+current_time+'.xlsx'
#build path to output file
output_file_path_1=os.path.join(input_directory,output_file_name_1)
output_file_path_2=os.path.join(input_directory,output_file_name_2)
#export file to excel from pandas
data_NAM.to_excel(output_file_path_1)
data_SUKEP.to_excel(output_file_path_2)


C:\Users\Filip.Mazurkiewicz\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
